# First Step: Clean data as indicated in paper

In [77]:
import pandas as pd

# load data
df = pd.read_csv('main_data.csv')
print(len(df))

19850


The data contains 19,850 individuals.

In [78]:
# Drop individuals with invalid birth dates
df = df.dropna(subset=['yob'])
# drop individuals with birth dates before 1996 and after 2001
df = df[(df.yob > 1996) & (df.yob <= 2003)]
# Probabaly also individuals were dropped with birth dates later than June 2003 and earlier than July 1997 because of German school entry cutoffs 
df = df[~((df.yob == 2003) & (df.mob > 6))]
df = df[~((df.yob == 1996) & (df.mob < 7))]
print(df['yob'].value_counts())
print(f"Number of observations left: {len(df)}")


yob
2001.0    3782
2002.0    3764
2000.0    3533
1999.0    2860
1998.0    2409
2003.0    1535
1997.0    1230
Name: count, dtype: int64
Number of observations left: 19113


Jan Marcus et al. had 19,095 participants left after performing this operation. My number deviates by 18.

In [79]:
# delete observations from people that attended third grade in another state
# List of values to keep
values = ["16. Thüringen", "13. Sachsen", "4. Brandenburg"]

# Only keep rows in df where 'bula_3rd' is in `values`
df = df[df.bula_3rd.isin(values)]
print(len(df))

17651


In [80]:
# disregard third graders that stated they attended third grade in year 2005, 2012, 2013 or 2014
df = df[(df['year_3rd']!="2004.0") & (df['year_3rd']!="2005.0") & (df['year_3rd']!="2006.0") & (df['year_3rd']!="2012.0") & (df['year_3rd']!="2013.0") & (df['year_3rd']!="2014.0")]
print(len(df))
print(df['year_3rd'].value_counts())
print(df['bula_3rd'].value_counts())


17002
year_3rd
2010/11    3479
2009/10    3323
2008/09    3011
2011/12    2897
2007/08    2410
2006/07    1708
Name: count, dtype: int64
bula_3rd
13. Sachsen       9334
4. Brandenburg    4661
16. Thüringen     3007
Name: count, dtype: int64


In [81]:
# drop individuals with missing treatment status
df = df.dropna(subset=['treat'])

In [82]:
# Disregard people that did not answer the 6 important questions for outcome variables
columns = ["kommheard", "kommgotten", "kommused", "sportsclub", "sport_hrs", "oweight"]
df = df.dropna(subset=columns)
print(len(df))


16082


In [83]:
# Disregard third graders in school year 2011/12
df = df[df['year_3rd'] != "2011/12"] 
print(len(df))
df.to_csv('cleaned_data.csv', index=False)

13334


# Data Cleaning Summary
- I was able to reproduce the cleaning steps by Jan Marcus et al.
- My cleaned data consists of 13,334 people. This exact number is also obtained by the authors.
- They might have applied the cleaning steps in slightly different order, as intermediate results have minor deviations

In [84]:
df['bula_3rd'].value_counts()

bula_3rd
13. Sachsen       7211
4. Brandenburg    3762
16. Thüringen     2361
Name: count, dtype: int64

# Checking treatment and control groups

In [85]:
# find number of saxons treated
treated = df[(df['treat']==1)]
print(f"Number of Saxons treated: {len(treated)}")

# untreated
untreated = df[(df['treat']==0)]
print(f"Number untreated: {len(untreated)}")

Number of Saxons treated: 5027
Number untreated: 8307


In [86]:
# find number of saxons pre-treatment
pre_treated = df[(df['treat']==0) & (df['bula_3rd'] == "13. Sachsen") & ((df['year_3rd'] == "2007/08") | (df['year_3rd'] == "2006/07"))]
print(f"Number of Saxons pre-treatment: {len(pre_treated)}")


Number of Saxons pre-treatment: 2184


In [87]:
# find number of control individuals during treatment
control = df[(df['treat']==0) & (df['bula_3rd'] != "13. Sachsen") & ((df['year_3rd'] == "2008/09") | (df['year_3rd'] == "2009/10") | (df['year_3rd'] == "2010/11"))]
print(f"Number of control individuals during treatment: {len(control)}")


Number of control individuals during treatment: 4344


In [88]:
# find number of control individuals pre-treatment
pre_control = df[(df['treat']==0) & (df['bula_3rd'] != "13. Sachsen") & ((df['year_3rd'] == "2007/08") | (df['year_3rd'] == "2006/07"))]
print(f"Number of control individuals pre-treatment: {len(pre_control)}")

Number of control individuals pre-treatment: 1779


Luckily, all the numbers add up and I can proceed

# Transform variable values to binary or int 

In [89]:
import numpy as np

In [90]:
# convert female to binary
print(df["female"].value_counts())
df["female"] = np.where(df["female"] == "1. weiblich", 1, 0)
print(df["female"].value_counts())

female
1. weiblich    7609
0. männlich    5722
Name: count, dtype: int64
female
1    7609
0    5725
Name: count, dtype: int64


In [91]:
# convert siblings to binary
print(df['siblings'].value_counts())
df['siblings'] = np.where(df['siblings'] == "1. Ja", 1, 0)
print(df['siblings'].value_counts())

siblings
1. Ja      11278
0. Nein     1907
Name: count, dtype: int64
siblings
1    11278
0     2056
Name: count, dtype: int64


In [92]:
# convert born_germany
print(df['born_germany'].value_counts())
df['born_germany'] = np.where(df['born_germany'] == "1. In Deutschland", 1, 0)
print(df['born_germany'].value_counts())

born_germany
1. In Deutschland                         12765
0. Außerhalb Deutschlands, und zwar in      493
Name: count, dtype: int64
born_germany
1    12765
0      569
Name: count, dtype: int64


In [93]:
# convert newspaper to binary
print(df['newspaper'].value_counts())
df['newspaper'] = np.where(df['newspaper'] == "1. Ja", 1, 0)
print(df['newspaper'].value_counts())

newspaper
1. Ja      7666
0. Nein    5576
Name: count, dtype: int64
newspaper
1    7666
0    5668
Name: count, dtype: int64


In [94]:
# convert art_at_home to binary
print(df['art_at_home'].value_counts())
df['art_at_home'] = np.where(df['art_at_home'] == "1. Ja", 1, 0)
print(df['art_at_home'].value_counts())

art_at_home
1. Ja      9643
0. Nein    3612
Name: count, dtype: int64
art_at_home
1    9643
0    3691
Name: count, dtype: int64


In [95]:
# save preprocessed data
df.to_csv('preprocessed_data.csv', index=False)